In [44]:
import pandas as pd

In [45]:
# Chequear que este en el Drive de los dos integrantes del grupo
file_path_dir = '../sandbox/football/kaggle/'
file_path_appearances = file_path_dir + 'appearances.csv'
file_path_players = file_path_dir + 'players.csv'
file_path_games = file_path_dir + 'games.csv'
file_path_player_valuations = file_path_dir + 'player_valuations.csv'

try:
  appearances = pd.read_csv(file_path_appearances)
  players = pd.read_csv(file_path_players)
  games = pd.read_csv(file_path_games)
  valuations = pd.read_csv(file_path_player_valuations)
  print("CSV loaded successfully.")
except FileNotFoundError:
  print("File not found at the specified path.")
except Exception as e:
  print("An error occurred while loading the CSV:", e)

CSV loaded successfully.


In [46]:
player_ids_names = players[["player_id", "name"]]
player_valuations_data = valuations[["player_id", "market_value_in_eur", "date"]]
players_with_age = players[["player_id", "name", "date_of_birth"]]

# drop any rows with missing values
player_ids_names = player_ids_names.dropna()
player_valuations_data = player_valuations_data.dropna()
players_with_age = players_with_age.dropna()

# reset the index of the DataFrames
player_ids_names = player_ids_names.reset_index(drop=True)
player_valuations_data = player_valuations_data.reset_index(drop=True)
players_with_age = players_with_age.reset_index(drop=True)

In [47]:
players_with_age['date_of_birth'] = players_with_age['date_of_birth'].astype('datetime64[ns]')

In [48]:
def calculate_age(born, date):
    return date.year - born.year - ((date.month, date.day) < (born.month, born.day))

In [49]:
def calculate_age(born, date):
    return date.year - born.year - ((date.month, date.day) < (born.month, born.day))

merged_df = players_with_age.merge(player_valuations_data, on='player_id').merge(player_ids_names, on='player_id')
merged_df['date'] = merged_df['date'].astype('datetime64[ns]')
merged_df

,player_id,name_x,date_of_birth,market_value_in_eur,date,name_y
0,10,Miroslav Klose,1978-06-09,7000000,2004-10-04,Miroslav Klose
1,10,Miroslav Klose,1978-06-09,9000000,2005-01-07,Miroslav Klose
2,10,Miroslav Klose,1978-06-09,12000000,2005-05-05,Miroslav Klose
3,10,Miroslav Klose,1978-06-09,15000000,2005-09-30,Miroslav Klose
4,10,Miroslav Klose,1978-06-09,20000000,2006-01-09,Miroslav Klose
...,...,...,...,...,...,...
487380,1294049,Tomás Silva,2006-05-25,100000,2024-10-08,Tomás Silva
487381,1294052,João Barros,2006-01-10,150000,2024-10-08,João Barros
487382,1296876,Marc Domènech,2006-12-01,500000,2024-10-11,Marc Domènech
487383,1302421,João Neto,2003-02-04,100000,2024-09-12,João Neto


In [50]:
merged_df['age'] = merged_df.apply(lambda x: calculate_age(x['date_of_birth'], x['date']), axis=1)
merged_df['name'] = merged_df['name_x']
merged_df = merged_df.drop(columns=['date_of_birth', 'date', 'name_x', 'name_y'])
merged_df = merged_df[['player_id', 'name', 'age', 'market_value_in_eur']]
merged_df

,player_id,name,age,market_value_in_eur
0,10,Miroslav Klose,26,7000000
1,10,Miroslav Klose,26,9000000
2,10,Miroslav Klose,26,12000000
3,10,Miroslav Klose,27,15000000
4,10,Miroslav Klose,27,20000000
...,...,...,...,...
487380,1294049,Tomás Silva,18,100000
487381,1294052,João Barros,18,150000
487382,1296876,Marc Domènech,17,500000
487383,1302421,João Neto,21,100000


In [51]:
merged_df.to_csv('players_age_valuation.csv', index=False)